# Symmetry, Lie Theory & Classical Integrability

**Theme: Solvability is Symmetry**

## 1. Introduction: The Algebraic Miracle

In ep. 1, we learned specific tricks to solve equations (Bernoulli substitutions, integrating factors). In ep. 3, we learned that physics is geometric. Now, in ep. 4, we discover that **Geometry is ruled by Algebra**.

We ask a fundamental question: *Why* are some equations exactly solvable while others are chaotic? The answer lies in **Symmetry**. When a differential equation admits a continuous group of transformations that map solutions to solutions (a Lie Group), we can use that symmetry to reduce the order of the equation or find conservation laws.

In rare, "miraculous" cases, a system possesses *infinite* symmetry. These are **Completely Integrable Systems**, where nonlinear waves (Solitons) behave like fundamental particles, passing through each other without scattering.


## 2. Lie Symmetries & The Prolongation Formula

Sophus Lie (1880) realized that integration techniques were just specific instances of invariance under group actions.

### 2.1 The Infinitesimal Generator

Consider a one-parameter group of transformations on the plane:

$$(x, y) \mapsto (\tilde{x}(x, y; \epsilon), \tilde{y}(x, y; \epsilon))$$

We represent this group by its **Vector Field generator**:

$$V = \xi(x, y) \frac{\partial}{\partial x} + \eta(x, y) \frac{\partial}{\partial y}$$

### 2.2 Prolongation to Jet Space

A differential equation involves derivatives $y'$, $y''$, etc. To apply the symmetry to the equation, we must extend ("prolong") the action of the group to the derivatives. The **Prolonged Vector Field** acts on the **Jet Space** $(x, y, y', y'', \ldots)$:

$$\text{pr}^{(n)} V = V + \eta^{(1)} \frac{\partial}{\partial y'} + \eta^{(2)} \frac{\partial}{\partial y''} + \cdots$$

The coefficients $\eta^{(k)}$ are determined recursively (e.g., $\eta^{(1)} = D_x(\eta) - y' D_x(\xi)$).

### 2.3 The Symmetry Condition

A differential equation $F(x, y, y', \ldots) = 0$ admits the symmetry $V$ if:

$$\text{pr}^{(n)} V[F] = 0 \quad \text{whenever } F = 0$$

Solving this linear PDE (the "Determining Equation") gives us the symmetries $V$. Once found, we can use "Canonical Coordinates" to reduce the order of the ODE by one, effectively "dividing" the differential equation by the symmetry group.


In [1]:
# [PLOT PLACEHOLDER: Lie Group Flow. A widget showing a direction field dy/dx = f(x,y) and a superimposed vector field V. Users can animate the flow of the plane along V to see that the integral curves of the ODE are permuted but not destroyed.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class LieGroupFlowWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['epsilon'] = ParameterSlider(
            name='epsilon',
            value=0.0,
            min_val=-1.0,
            max_val=1.0,
            step=0.1,
            description='Group Parameter ε'
        )
        self.sliders['ode_type'] = ParameterSlider(
            name='ode_type',
            value=0,
            min_val=0,
            max_val=1,
            step=1,
            description='ODE Type (0: Linear, 1: Nonlinear)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            eps = params['epsilon']
            ode_type = int(params['ode_type'])

            # Grid for direction field
            x = np.linspace(-2, 2, 25)
            y = np.linspace(-2, 2, 25)
            X, Y = np.meshgrid(x, y)

                        # ODE direction field dy/dx = f(x,y)
            if ode_type == 0:
                f = Y - X  # Linear: y' = y - x
                title = 'Linear ODE'
            else:
                f = Y**2 - X  # Nonlinear: y' = y² - x
                title = 'Nonlinear ODE'

            U_ode = np.ones_like(X)
            V_ode = f

            # Normalize for arrows
            norm_ode = np.sqrt(U_ode**2 + V_ode**2) + 1e-10
            U_ode_norm = U_ode / norm_ode
            V_ode_norm = V_ode / norm_ode

            # Symmetry generator V = ξ ∂x + η ∂y, e.g., scaling symmetry
            # For scaling: V = x ∂x + y ∂y
            Xi = X
            Eta = Y

            # Flow the plane along V: transformed (x,y) = exp(ε V)(x,y)
            # For scaling symmetry V = x ∂x + y ∂y: x(ε) = x e^ε, y(ε) = y e^ε
            X_trans = X * np.exp(eps)
            Y_trans = Y * np.exp(eps)

            # CRITICAL: The ODE field must be evaluated at the TRANSFORMED coordinates
            # This shows how the direction field "moves" with the transformation
            if ode_type == 0:
                f_trans = Y_trans - X_trans  # Evaluate ODE at transformed points
            else:
                f_trans = Y_trans**2 - X_trans

            U_ode_trans = np.ones_like(X_trans)
            V_ode_trans = f_trans

            # Normalize transformed ODE field
            norm_ode_trans = np.sqrt(U_ode_trans**2 + V_ode_trans**2) + 1e-10
            U_ode_trans_norm = U_ode_trans / norm_ode_trans
            V_ode_trans_norm = V_ode_trans / norm_ode_trans

            # Create figure with dynamic range based on transformation
            max_range = max(2.5, 2.5 * np.exp(abs(eps)))
            fig = self.plot_manager.create_plotly_figure(width=800, height=700)

            # Create vector field arrows (ODE field at ORIGINAL positions) - reference
            skip = 3
            scale_ode = 0.2
            arrow_x_ode = []
            arrow_y_ode = []
            for i in range(0, U_ode_norm.shape[0], skip):
                for j in range(0, U_ode_norm.shape[1], skip):
                    xi, yi = X[i, j], Y[i, j]
                    ui, vi = U_ode_norm[i, j] * scale_ode, V_ode_norm[i, j] * scale_ode
                    arrow_x_ode.extend([xi, xi + ui, None])
                    arrow_y_ode.extend([yi, yi + vi, None])

            fig.add_trace(
                go.Scatter(
                    x=arrow_x_ode,
                    y=arrow_y_ode,
                    mode='lines',
                    line=dict(color=self.plot_manager.config.colors[1], width=1.5),
                    name='ODE Field (Original)',
                    showlegend=True,
                    opacity=0.6
                )
            )

            # CRITICAL: Show ODE field at TRANSFORMED positions - this is what moves!
            arrow_x_ode_trans = []
            arrow_y_ode_trans = []
            for i in range(0, U_ode_trans_norm.shape[0], skip):
                for j in range(0, U_ode_trans_norm.shape[1], skip):
                    xi, yi = X_trans[i, j], Y_trans[i, j]
                    ui, vi = U_ode_trans_norm[i, j] * scale_ode, V_ode_trans_norm[i, j] * scale_ode
                    # Only show if in view
                    if abs(xi) <= max_range and abs(yi) <= max_range:
                        arrow_x_ode_trans.extend([xi, xi + ui, None])
                        arrow_y_ode_trans.extend([yi, yi + vi, None])

            fig.add_trace(
                go.Scatter(
                    x=arrow_x_ode_trans,
                    y=arrow_y_ode_trans,
                    mode='lines',
                    line=dict(color=self.plot_manager.config.colors[0], width=2),
                    name='ODE Field (Transformed)',
                    showlegend=True
                )
            )

            # Symmetry field (red) - also on original grid
            norm_sym = np.sqrt(Xi**2 + Eta**2) + 1e-10
            Xi_norm = Xi / norm_sym
            Eta_norm = Eta / norm_sym
            scale_sym = 0.15
            arrow_x_sym = []
            arrow_y_sym = []
            for i in range(0, Xi_norm.shape[0], skip):
                for j in range(0, Xi_norm.shape[1], skip):
                    xi, yi = X[i, j], Y[i, j]
                    ui, vi = Xi_norm[i, j] * scale_sym, Eta_norm[i, j] * scale_sym
                    arrow_x_sym.extend([xi, xi + ui, None])
                    arrow_y_sym.extend([yi, yi + vi, None])

            fig.add_trace(
                go.Scatter(
                    x=arrow_x_sym,
                    y=arrow_y_sym,
                    mode='lines',
                    line=dict(color=self.plot_manager.config.colors[3], width=2),
                    name='Symmetry V',
                    showlegend=True,
                    opacity=0.7
                )
            )

            # Transformed grid points (green markers) - show where original points moved to
            # Use a sparser grid for clarity
            grid_skip = skip * 2
            x_trans_flat = X_trans[::grid_skip, ::grid_skip].flatten()
            y_trans_flat = Y_trans[::grid_skip, ::grid_skip].flatten()

            # Only show points that are still in view
            in_view = (np.abs(x_trans_flat) <= max_range) & (np.abs(y_trans_flat) <= max_range)

            fig.add_trace(
                go.Scatter(
                    x=x_trans_flat[in_view],
                    y=y_trans_flat[in_view],
                    mode='markers',
                    marker=dict(size=6, color=self.plot_manager.config.colors[2], symbol='circle'),
                    name='Transformed Grid',
                    showlegend=True
                )
            )

            # Also show original grid points for comparison (when eps != 0)
            if abs(eps) > 0.01:
                x_orig_flat = X[::grid_skip, ::grid_skip].flatten()
                y_orig_flat = Y[::grid_skip, ::grid_skip].flatten()
                fig.add_trace(
                    go.Scatter(
                        x=x_orig_flat,
                        y=y_orig_flat,
                        mode='markers',
                        marker=dict(size=4, color='gray', symbol='x', opacity=0.5),
                        name='Original Grid',
                        showlegend=True
                    )
                )

            fig.update_layout(
                title=f'Lie Symmetry Flow (ε={eps:.1f}) - {title}<br>ODE Direction Field Transforms with Grid (Integral Curves Preserved)',
                xaxis_title='$x$',
                yaxis_title='$y$',
                xaxis=dict(scaleanchor="y", scaleratio=1, range=[-max_range, max_range]),
                yaxis=dict(range=[-max_range, max_range])
            )

            fig.show()

widget = LieGroupFlowWidget(title="Lie Group Flow on ODE Direction Field")
widget.display()


## 3. Noether's Theorem

For physical systems derived from a variational principle, symmetry implies something even stronger: Conservation.

### 3.1 The Action Principle

Let an equation arise from minimizing an action $S = \int L(q, \dot{q}, t) dt$.

### 3.2 The Theorem

**Noether's Theorem (1918):** If the action functional is invariant under a continuous symmetry group generated by $V$, then there exists a conserved quantity $Q$ (constant in time):

$$\frac{dQ}{dt} = 0$$

This connects abstract algebra to concrete physics:

* Time Translation $\to$ **Conservation of Energy**

* Spatial Translation $\to$ **Conservation of Momentum**

* Rotational Invariance $\to$ **Conservation of Angular Momentum**

* Gauge Invariance $\to$ **Conservation of Charge**


In [2]:
# [PLOT PLACEHOLDER: Noether's Machine. An interactive Lagrangian system (e.g., a pendulum). Users select a symmetry (e.g., rotation) and the widget plots the corresponding quantity (angular momentum) in real-time to check for conservation.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider
from diffeq.core.solvers import ODESolver

class NoethersMachineWidget(InteractiveWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._ref_eigenvalues = None  # For spectral widget compatibility

    def _setup_widgets(self):
        self.sliders['symmetry_type'] = ParameterSlider(
            name='symmetry_type',
            value=0,
            min_val=0,
            max_val=1,
            step=1,
            description='Symmetry (0: Time, 1: Angular)'
        )
        self.sliders['initial_angle'] = ParameterSlider(
            name='initial_angle',
            value=45.0,
            min_val=0.0,
            max_val=90.0,
            step=5.0,
            description='Initial Angle (°)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            sym_type = int(params['symmetry_type'])
            theta0 = params['initial_angle'] * np.pi / 180

            # Pendulum: Lagrangian L = (1/2) m l² θ'² - m g l (1 - cos θ)
            # Equations: θ'' + (g/l) sin θ = 0
            g = 9.81
            l = 1.0
            t = np.linspace(0, 10, 500)

            # Solve pendulum ODE inline
            def pendulum_rhs(t_val, y):
                theta, omega = y[0], y[1]
                return np.array([omega, -(g/l) * np.sin(theta)])

            t_span = (t[0], t[-1])
            _, y = ODESolver.solve_ivp(pendulum_rhs, t_span, np.array([theta0, 0.0]), t_eval=t, method="DOP853")
            theta = y[:, 0]
            omega = y[:, 1]

            # Conserved quantities
            if sym_type == 0:
                # Time symmetry → Energy E = (1/2) l² ω² + g l (1 - cos θ)
                conserved = 0.5 * l**2 * omega**2 + g * l * (1 - np.cos(theta))
                title = 'Time Symmetry - Energy Conservation'
                ylabel = 'Energy $E$'
            else:
                # Angular momentum p_θ = l² ω (conserved if no explicit time dep)
                conserved = l**2 * omega
                title = 'Angular Momentum Conservation'
                ylabel = 'Angular Momentum $p_\\theta$'

            fig = make_subplots(
                rows=2, cols=1,
                subplot_titles=('Angle $\\theta(t)$', title),
                vertical_spacing=0.15,
                row_heights=[0.5, 0.5]
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=700,
                width=1000,
                showlegend=True
            )

            # Plot theta(t)
            fig.add_trace(
                go.Scatter(
                    x=t, y=theta,
                    mode='lines',
                    name='$\\theta(t)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                ),
                row=1, col=1
            )

            # Plot conserved quantity
            fig.add_trace(
                go.Scatter(
                    x=t, y=conserved,
                    mode='lines',
                    name=ylabel,
                    line=dict(color=self.plot_manager.config.colors[3], width=2)
                ),
                row=2, col=1
            )

            # Add horizontal line showing constancy
            mean_conserved = np.mean(conserved)
            std_conserved = np.std(conserved)
            rel_variation = std_conserved / (abs(mean_conserved) + 1e-10) * 100

            fig.add_hline(
                y=mean_conserved,
                line_dash="dash",
                line_color="gray",
                annotation_text=f'Mean: {mean_conserved:.4f} (σ={std_conserved:.6f}, {rel_variation:.3f}% variation)',
                row=2, col=1
            )

            # Add shaded region showing ±1σ to emphasize constancy
            fig.add_hrect(
                y0=mean_conserved - std_conserved,
                y1=mean_conserved + std_conserved,
                fillcolor="gray",
                opacity=0.2,
                layer="below",
                line_width=0,
                row=2, col=1
            )

            fig.update_xaxes(title_text="Time $t$", row=2, col=1)
            fig.update_yaxes(title_text="$\\theta(t)$", row=1, col=1)
            fig.update_yaxes(title_text=ylabel, row=2, col=1)

            fig.show()

widget = NoethersMachineWidget(title="Noether's Machine: Symmetry and Conservation")
widget.display()


## 4. Completely Integrable Systems & Solitons

Some nonlinear PDEs possess not just one or two, but **infinite** conservation laws. The prototype is the **Korteweg-de Vries (KdV)** equation for shallow water waves:

$$u_t + u u_x + u_{xxx} = 0$$

### 4.1 Solitons

The KdV equation supports **Solitons**: localized wave packets that maintain their shape while propagating. Remarkably, when two solitons collide, they interact nonlinearly but emerge unchanged, preserving their identity like particles. This behavior is impossible in generic nonlinear systems (where waves would shatter or merge).


In [ ]:
# [PLOT PLACEHOLDER: The Soliton Collider. A simulation of the KdV equation. Users launch two solitons of different amplitudes/speeds. They collide, undergo a complex nonlinear phase shift, and separate cleanly.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class SolitonColliderWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['amp1'] = ParameterSlider(
            name='amp1',
            value=2.0,
            min_val=1.0,
            max_val=5.0,
            step=0.5,
            description='Amplitude 1'
        )
        self.sliders['amp2'] = ParameterSlider(
            name='amp2',
            value=1.0,
            min_val=0.5,
            max_val=3.0,
            step=0.5,
            description='Amplitude 2'
        )
        self.sliders['time'] = ParameterSlider(
            name='time',
            value=0.0,
            min_val=0.0,
            max_val=10.0,
            step=0.1,
            description='Time $t$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            a1 = params['amp1']
            a2 = params['amp2']
            t = params['time']

            # KdV solitons: u(x,t) = (c/2) sech²(√(c/4) (x - c t - x0)), c = speed ~ amp
            c1 = 4 * a1  # Speed proportional to amp
            c2 = 4 * a2
            x01 = -10
            x02 = 5

            # Calculate dynamic range to capture full collision
            # Soliton positions: x1(t) = x01 + c1*t, x2(t) = x02 + c2*t
            # Need range that covers both solitons throughout the time span
            t_max = 10.0
            x1_min = x01  # At t=0
            x1_max = x01 + c1 * t_max  # At t=t_max
            x2_min = x02  # At t=0
            x2_max = x02 + c2 * t_max  # At t=t_max

            # Add padding for soliton width (solitons have width ~ 2/√(c/4) = 4/√c)
            width1 = 4.0 / np.sqrt(c1) if c1 > 0 else 2.0
            width2 = 4.0 / np.sqrt(c2) if c2 > 0 else 2.0
            padding = max(width1, width2) * 2

            x_min = min(x1_min, x2_min) - padding
            x_max = max(x1_max, x2_max) + padding

            x = np.linspace(x_min, x_max, 800)  # Higher resolution for better visualization

            # Single soliton solutions
            soliton1 = (c1 / 2) * (1 / np.cosh(np.sqrt(c1 / 4) * (x - c1 * t - x01)))**2
            soliton2 = (c2 / 2) * (1 / np.cosh(np.sqrt(c2 / 4) * (x - c2 * t - x02)))**2

            # Two-soliton solution: superposition when well-separated, interaction during collision
            # For KdV, solitons interact nonlinearly but emerge unchanged (phase shift only)
            u_approx = soliton1 + soliton2

            # During collision, account for nonlinear interaction
            # The peak during collision is higher than simple sum due to constructive interference
            overlap_mask = (soliton1 > 0.05) & (soliton2 > 0.05)
            if np.any(overlap_mask):
                # Enhanced interaction: peak is boosted during collision
                # This is a simplified model - exact solution has phase shift
                interaction_factor = 1.0 + 0.4 * np.exp(-((t - 3.75)**2) / 0.5)  # Peak interaction near collision
                u_approx[overlap_mask] = (soliton1[overlap_mask] + soliton2[overlap_mask]) * interaction_factor

                # Ensure smooth transition
                transition = np.exp(-((soliton1[overlap_mask] - soliton2[overlap_mask])**2) / 0.1)
                u_approx[overlap_mask] = (1 - transition) * u_approx[overlap_mask] + transition * (soliton1[overlap_mask] + soliton2[overlap_mask])

            fig = self.plot_manager.create_plotly_figure(width=900, height=600)

            # Get color for fill (use same color as line with opacity)
            line_color = self.plot_manager.config.colors[0]
            fig.add_trace(
                go.Scatter(
                    x=x, y=u_approx,
                    mode='lines',
                    name='$u(x,t)$',
                    line=dict(color=line_color, width=2),
                    fill='tozeroy',
                    fillcolor=line_color,
                    opacity=0.3
                )
            )

            # Add individual solitons (faded) for reference - show their trajectories
            fig.add_trace(
                go.Scatter(
                    x=x, y=soliton1,
                    mode='lines',
                    name=f'Soliton 1 (c={c1:.1f})',
                    line=dict(color=self.plot_manager.config.colors[1], width=1.5, dash='dot'),
                    opacity=0.6
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=x, y=soliton2,
                    mode='lines',
                    name=f'Soliton 2 (c={c2:.1f})',
                    line=dict(color=self.plot_manager.config.colors[2], width=1.5, dash='dot'),
                    opacity=0.6
                )
            )

            # Mark soliton centers for clarity
            x1_center = x01 + c1 * t
            x2_center = x02 + c2 * t
            fig.add_trace(
                go.Scatter(
                    x=[x1_center, x2_center],
                    y=[soliton1[np.argmin(np.abs(x - x1_center))], soliton2[np.argmin(np.abs(x - x2_center))]],
                    mode='markers',
                    marker=dict(size=8, color=self.plot_manager.config.colors[1], symbol='circle'),
                    name='Soliton Centers',
                    showlegend=False,
                    hoverinfo='skip'
                )
            )

            fig.update_layout(
                title=f'KdV Soliton Collision at $t={t:.1f}$<br>Amplitudes: {a1:.1f}, {a2:.1f} (Speeds: {c1:.1f}, {c2:.1f})',
                xaxis_title='$x$',
                yaxis_title='$u(x,t)$',
                xaxis_range=[x_min, x_max],
                yaxis_range=[0, max(a1, a2) + 1],
                showlegend=True
            )

            fig.show()

widget = SolitonColliderWidget(title="KdV Soliton Collider")
widget.display()


## 5. The Lax Pair & Inverse Scattering

How can a nonlinear equation be exactly solvable? The answer lies in disguise: it is actually a linear equation in a higher dimension.

### 5.1 The Lax Pair

Peter Lax (1968) showed that the KdV equation is equivalent to the compatibility condition of two **Linear Operators**, $L$ and $M$:

$$L_t = [M, L] = ML - LM$$

The compatibility condition $L_t = [M, L]$ forces the operator equation:

$$\frac{\partial L}{\partial t} = [M, L]$$

If we choose $L = -\frac{d^2}{dx^2} + u$ (the Schrödinger operator with potential $u$), this commutator equation reproduces KdV.

### 5.2 Iso-spectral Flow

Crucially, the eigenvalues $\lambda_k$ of $L$ are **constant in time** ($\frac{d\lambda_k}{dt} = 0$). The nonlinear wave evolution $u(x,t)$ is just a unitary rotation of the operator $L$.

### 5.3 The Inverse Scattering Transform (IST)

This leads to a "Nonlinear Fourier Transform" to solve the initial value problem:

1. **Direct Scattering:** Map initial data $u(x,0)$ to its spectral data (eigenvalues/reflection coefficients of $L$).

2. **Time Evolution:** Evolve the spectral data linearly (trivial phase rotation).

3. **Inverse Scattering:** Reconstruct $u(x,t)$ from the evolved spectral data using the **Gelfand-Levitan-Marchenko (GLM)** integral equation.


In [4]:
# [PLOT PLACEHOLDER: Spectral Monitor. A dual plot. Top: The evolving KdV wave u(x,t). Bottom: The discrete eigenvalues of the Schrödinger operator L = -d²/dx² + u. As the wave sloshes violently, the eigenvalues remain perfectly frozen.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.linalg import eigh

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class SpectralMonitorWidget(InteractiveWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._ref_eigenvalues = None  # Store reference eigenvalues at t=0

    def _setup_widgets(self):
        self.sliders['time'] = ParameterSlider(
            name='time',
            value=0.0,
            min_val=0.0,
            max_val=5.0,
            step=0.1,
            description='Time $t$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            t = params['time']

            # KdV evolution with initial u0 = sech²(x) (single soliton)
            x = np.linspace(-10, 10, 200)
            dx = x[1] - x[0]

            # Soliton solution: u(x,t) = 2 sech²(x - 4t - x0)
            x0 = 0.0
            u = 2.0 / np.cosh(x - 4*t - x0)**2

            # Schrödinger operator L = -d²/dx² + u, discrete eigenvalues
            # Finite difference matrix for -d²/dx²
            n = len(x)
            diag_main = -2.0 / dx**2 * np.ones(n)
            diag_upper = 1.0 / dx**2 * np.ones(n-1)
            diag_lower = 1.0 / dx**2 * np.ones(n-1)

            # Add potential u
            L = np.diag(diag_main + u) + np.diag(diag_upper, 1) + np.diag(diag_lower, -1)

            # Compute eigenvalues (only negative ones are bound states)
            evals = eigh(L, eigvals_only=True)
            bound_states = np.sort(evals[evals < 0])[:5]  # Up to 5 bound states, sorted

                        # Store reference eigenvalues (at t=0) for comparison
            if self._ref_eigenvalues is None or t == 0.0:
                self._ref_eigenvalues = bound_states.copy()

            # Compare with reference to emphasize constancy
            if len(bound_states) > 0 and self._ref_eigenvalues is not None and len(self._ref_eigenvalues) > 0:
                min_len = min(len(bound_states), len(self._ref_eigenvalues))
                eigenvalue_drift = np.abs(bound_states[:min_len] - self._ref_eigenvalues[:min_len])
                max_drift = np.max(eigenvalue_drift) if len(eigenvalue_drift) > 0 else 0.0
            else:
                max_drift = 0.0

            fig = make_subplots(
                rows=2, cols=1,
                subplot_titles=('KdV Wave $u(x,t)$', 'Eigenvalues $\\lambda_k$ of $L = -\\frac{d^2}{dx^2} + u$'),
                vertical_spacing=0.15,
                row_heights=[0.6, 0.4]
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=800,
                width=1000,
                showlegend=True
            )

            # Top: u(x,t)
            fig.add_trace(
                go.Scatter(
                    x=x, y=u,
                    mode='lines',
                    name='$u(x,t)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=2),
                    fill='tozeroy'
                ),
                row=1, col=1
            )

            # Bottom: eigenvalues - emphasize constancy
            if len(bound_states) > 0:
                k_indices = np.arange(len(bound_states))

                # Current eigenvalues
                fig.add_trace(
                    go.Scatter(
                        x=k_indices,
                        y=bound_states,
                        mode='markers+lines',
                        name=f'Eigenvalues $\\lambda_k$ (t={t:.1f})',
                        marker=dict(size=12, color=self.plot_manager.config.colors[3], symbol='circle'),
                        line=dict(color=self.plot_manager.config.colors[3], width=3)
                    ),
                    row=2, col=1
                )

                # Reference eigenvalues (t=0) for comparison
                if self._ref_eigenvalues is not None and len(self._ref_eigenvalues) > 0:
                    ref_k = np.arange(len(self._ref_eigenvalues))
                    fig.add_trace(
                        go.Scatter(
                            x=ref_k,
                            y=self._ref_eigenvalues,
                            mode='markers',
                            name='Reference (t=0)',
                            marker=dict(size=8, color='white', symbol='x', line=dict(width=2, color=self.plot_manager.config.colors[3])),
                            opacity=0.8
                        ),
                        row=2, col=1
                    )

                    # Add annotation showing max drift
                    if max_drift < 1e-6:
                        drift_text = f'Max drift: < 1e-6 (Perfectly constant!)'
                    else:
                        drift_text = f'Max drift: {max_drift:.2e}'
            else:
                # No bound states
                fig.add_trace(
                    go.Scatter(
                        x=[0], y=[0],
                        mode='markers',
                        name='No bound states',
                        marker=dict(size=10, color=self.plot_manager.config.colors[3])
                    ),
                    row=2, col=1
                )

            fig.update_xaxes(title_text="$x$", row=1, col=1)
            fig.update_xaxes(title_text="$k$", row=2, col=1)
            fig.update_yaxes(title_text="$u(x,t)$", row=1, col=1)
            fig.update_yaxes(title_text="$\\lambda_k$", row=2, col=1)

            # Update title with drift information
            if len(bound_states) > 0 and max_drift < 1e-4:
                title_text = f'KdV Spectral Monitor at $t={t:.1f}$<br>Eigenvalues Remain Constant (Iso-spectral Flow) - Max Drift: {max_drift:.2e}'
            else:
                title_text = f'KdV Spectral Monitor at $t={t:.1f}$<br>Eigenvalues Remain Constant (Iso-spectral Flow)'

            fig.update_layout(title=title_text)

            fig.show()

widget = SpectralMonitorWidget(title="KdV Iso-spectral Evolution")
widget.display()


## 6. Supersymmetric Quantum Mechanics (SUSY QM)

We conclude by applying algebraic factorization to the Schrödinger equation itself.

### 6.1 Factorization

If a Hamiltonian can be written as $H_- = A^\dagger A$, there exists a "partner" Hamiltonian $H_+ = AA^\dagger$ that shares the exact same energy spectrum (except possibly the ground state).

This creates a hierarchy of solvable potentials (Shape Invariance).

### 6.2 Connection to Integrability

The connection between partner potentials is related to the Bäcklund transformations that generate solitons. This links Quantum Mechanics, Solitons, and Group Theory into a single "Integrable" framework.


In [5]:
# [PLOT PLACEHOLDER: SUSY Partnering. Select a potential V(x) (e.g., Pöschl-Teller). The widget computes the Superpotential W(x), displays the partner potential V+(x), and shows the alignment of their energy levels.]

import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

class SusyPartneringWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['potential_type'] = ParameterSlider(
            name='potential_type',
            value=0,
            min_val=0,
            max_val=1,
            step=1,
            description='Potential (0: Harmonic, 1: Pöschl-Teller)'
        )
        self.sliders['n'] = ParameterSlider(
            name='n',
            value=2,
            min_val=1,
            max_val=5,
            step=1,
            description='Quantum Number $n$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            pot_type = int(params['potential_type'])
            n = int(params['n'])

            x = np.linspace(-5, 5, 500)
            dx = x[1] - x[0]

            if pot_type == 0:
                # Harmonic: V- = (1/2) x²
                V_minus = 0.5 * x**2
                # Superpotential W = x / √2 (for ground state ψ₀ ~ exp(-x²/2))
                W = x / np.sqrt(2.0)
                title = 'Harmonic Oscillator'
                # Energy levels: E_k = k + 1/2
                energies_minus = np.array([0.5, 1.5, 2.5, 3.5, 4.5])
            else:
                # Pöschl-Teller: V- = -n(n+1) / cosh²(x)
                V_minus = -n * (n + 1) / np.cosh(x)**2
                # Superpotential W = n tanh(x)
                W = n * np.tanh(x)
                title = 'Pöschl-Teller'
                # Energy levels: E_k = -(n-k)² for k=0,...,n-1
                energies_minus = np.array([-(n-k)**2 for k in range(min(n, 5))])

            # Partner V+ = W² + W'
            W_prime = np.gradient(W, dx)
            V_plus = W**2 + W_prime

            # Partner energies (shifted by one level)
            energies_plus = energies_minus[1:] if len(energies_minus) > 1 else np.array([])

            fig = self.plot_manager.create_plotly_figure(width=900, height=700)

            # Potentials
            fig.add_trace(
                go.Scatter(
                    x=x, y=V_minus,
                    mode='lines',
                    name='$V_-(x)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )
            fig.add_trace(
                go.Scatter(
                    x=x, y=V_plus,
                    mode='lines',
                    name='$V_+(x)$',
                    line=dict(color=self.plot_manager.config.colors[1], width=2, dash='dash')
                )
            )

                        # Energy levels as horizontal lines with better visualization
            # V- energy levels
            for i, e in enumerate(energies_minus):
                fig.add_hline(
                    y=e,
                    line_dash='dot',
                    line_color=self.plot_manager.config.colors[0],
                    line_width=2,
                    opacity=0.7,
                    annotation_text=f'$E_{{{i}}}={e:.2f}$',
                    annotation_position='right',
                    annotation_font_size=10
                )

            # V+ energy levels (partner) - show alignment
            for i, e in enumerate(energies_plus):
                # Check if this aligns with a V- level (shifted by one)
                aligned = False
                if i + 1 < len(energies_minus):
                    # Should align with E_{i+1} of V-
                    if abs(e - energies_minus[i + 1]) < 0.01:
                        aligned = True

                fig.add_hline(
                    y=e,
                    line_dash='dot',
                    line_color=self.plot_manager.config.colors[1],
                    line_width=2,
                    opacity=0.7,
                    annotation_text=f'$E_{{{i+1}}}^+={e:.2f}$' + (' ✓' if aligned else ''),
                    annotation_position='left',
                    annotation_font_size=10
                )

            # Add connecting lines to show alignment (for first few levels)
            for i in range(min(3, len(energies_plus))):
                if i + 1 < len(energies_minus):
                    # Draw a subtle line connecting aligned levels
                    x_conn = [np.min(x), np.max(x)]
                    y_conn = [energies_minus[i + 1], energies_plus[i]]
                    if abs(energies_minus[i + 1] - energies_plus[i]) < 0.1:
                        fig.add_trace(
                            go.Scatter(
                                x=x_conn,
                                y=y_conn,
                                mode='lines',
                                line=dict(color='yellow', width=1, dash='dot'),
                                opacity=0.3,
                                showlegend=False,
                                hoverinfo='skip'
                            )
                        )

            # Calculate energy range for better visualization
            all_energies = np.concatenate([energies_minus, energies_plus]) if len(energies_plus) > 0 else energies_minus
            energy_min = np.min(all_energies) - 0.5
            energy_max = max(np.max(V_minus), np.max(V_plus)) + 1

            fig.update_layout(
                title=f'SUSY Partner Potentials: {title} ($n={n}$)<br>Energy Levels Aligned (Except Ground State)',
                xaxis_title='$x$',
                yaxis_title='$V(x)$',
                showlegend=True,
                yaxis_range=[energy_min, energy_max],
                xaxis_range=[np.min(x), np.max(x)]
            )

            fig.show()

widget = SusyPartneringWidget(title="SUSY Quantum Mechanics Partner Potentials")
widget.display()


---

### References

* **Bluman, G. W., & Cole, J. D.** (1974). *Similarity methods for differential equations*.

* **Korteweg, D. J., & de Vries, G.** (1895). *On the change of form of long waves advancing in a rectangular canal*.

* **Lax, P. D.** (1968). *Integrals of nonlinear equations of evolution and solitary waves*.

* **Lie, S.** (1880). *Theorie der Transformationsgruppen*.

* **Noether, E.** (1918). *Invariante Variationsprobleme*.

* **Olver, P. J.** (1986). *Applications of Lie Groups to Differential Equations*.

* **Witten, E.** (1981). *Dynamical breaking of supersymmetry*.

* **Zakharov, V. E., & Shabat, A. B.** (1972). *Exact theory of two-dimensional self-focusing and one-dimensional self-modulation of waves in nonlinear media*.
